# do analysis on same trials data saving for color/number model
# common trials here: the trials with the same distribution and same myCard number

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["OMP_NUM_THREADS"] = "1"
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
from functools import reduce


In [ ]:
output_dir = r"10_RL_agent_TDlearn_output_same_trials_data"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'



folder_path_greedy = "13_RL_agent_TDlearn_output/model_behavior"
folder_path_softmax = "13_RL_agent_TDlearn_output_softmax/model_behavior"
folder_path_rs = "13_RL_agent_TDlearn_output_risk_sensitive/model_behavior"
folder_path_wsls = "13_RL_agent_TDlearn_output_wsls/model_behavior"
folder_path_dualQ = "13_RL_agent_TDlearn_output_risk_dualQ/model_behavior"
folder_path_dualQ_no_eta = "13_RL_agent_TDlearn_output_risk_dualQ_no_eta/model_behavior"




df_participants = []
df_both_greedy = []
df_both_softmax = []
df_both_wsls = []
df_both_rs = []
df_both_dualQ = []
df_both_dualQ_no_eta = []
clean_names= []

def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True)  
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")
        clean_names.append(clean_name)


        find_matching_csv(folder_path_greedy, df_both_greedy)
        find_matching_csv(folder_path_softmax, df_both_softmax)
        find_matching_csv(folder_path_wsls, df_both_wsls)
        find_matching_csv(folder_path_rs, df_both_rs)
        find_matching_csv(folder_path_dualQ, df_both_dualQ)
        find_matching_csv(folder_path_dualQ_no_eta, df_both_dualQ_no_eta)
        

In [3]:

df_counts = []
for df, name in zip(df_participants, clean_names):
    counts = df.groupby(['myCard', 'distribution']).size().reset_index(name=f'count_{name}')
    df_counts.append(counts)

common_trials = reduce(lambda left, right: pd.merge(left, right, on=['myCard', 'distribution']), df_counts)

count_cols = [col for col in common_trials.columns if col.startswith('count_')]
common_trials['min_count'] = common_trials[count_cols].min(axis=1)


total_common_trials = common_trials['min_count'].sum()
print("Total number of common trials:", total_common_trials)
common_trials

Total number of common trials: 181


,myCard,distribution,count_07_11_2024_17_23_43,count_08_11_2024_13_03_29,count_11_11_2024_16_46_44,count_12_11_2024_00_15_17,count_13_11_2024_10_46_21,count_13_11_2024_14_45_52,count_14_11_2024_21_46_47,count_15_11_2024_11_43_48,...,count_25_11_2024_07_37_11,count_25_11_2024_12_11_10,count_25_11_2024_18_41_38,count_25_11_2024_20_12_41,count_26_03_2025_16_21_25,count_26_11_2024_10_53_23,count_26_11_2024_14_31_40,count_28_11_2024_12_21_16,count_28_11_2024_22_38_25,min_count
0,1,high,3,3,2,1,3,1,3,2,...,3,1,2,2,2,2,2,1,2,1
1,1,low,15,15,17,17,20,17,16,15,...,16,16,18,20,17,15,23,16,16,13
2,1,uniform,11,7,11,8,9,9,9,8,...,10,9,11,8,7,12,9,10,11,7
3,2,high,4,2,5,4,2,4,4,4,...,5,3,4,3,4,4,5,5,3,2
4,2,low,16,14,19,15,15,16,15,18,...,17,18,16,16,17,16,15,15,17,11
5,2,uniform,9,8,9,9,10,9,8,8,...,9,9,9,8,9,11,9,9,12,7
6,3,high,5,11,5,6,8,6,6,6,...,5,7,6,8,3,5,8,3,8,3
7,3,low,11,16,11,18,14,14,13,16,...,13,14,12,15,13,15,14,17,16,11
8,3,uniform,9,8,8,10,12,6,9,11,...,9,12,9,10,7,7,12,11,8,6
9,4,high,7,6,9,7,9,7,9,6,...,9,11,9,7,8,12,9,10,8,5


# adding some columns to models

In [4]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    


for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    participant_outcome = df_participants[i]['outcome']
    arrowRT = df_participants[i]['arrowRT']
    spaceRT = df_participants[i]['spaceRT']
    risk = df_participants[i]['risk']
    
    for df_list in [df_both_greedy, df_both_softmax, df_both_wsls, df_both_rs, df_both_dualQ, df_both_dualQ_no_eta]:
        df_list[i]['block'] = df_participants[i]['block']
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type
        df_list[i]['participant_outcome'] = participant_outcome
        df_list[i]['arrowRT'] = arrowRT
        df_list[i]['spaceRT'] = spaceRT
        df_list[i]['risk'] = risk




def calculate_outcomes(df_list):
    for df in df_list:
        outcomes = []
        for i in range(len(df)):
            my = df.loc[i, 'myCard']
            your = df.loc[i, 'yourCard']
            choice = df.loc[i, 'model_choices']

            if ((my > your and choice == 1) or (my < your and choice == 0)):
                outcomes.append('win')
            else:
                outcomes.append('lose')

        df['model_outcome'] = outcomes


for df_list in [df_both_greedy, df_both_softmax, df_both_wsls, df_both_rs, df_both_dualQ, df_both_dualQ_no_eta]:
    calculate_outcomes(df_list)


In [5]:
df_both_dualQ[1]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,block,myCard,yourCard,distribution,block_type,participant_outcome,arrowRT,spaceRT,risk,model_outcome
0,0,1,9.5,10.5,"[[[0.08202077598288503, -0.02283112974341891],...",1,7,4,uniform,uniform,win,702,5576,0.250,lose
1,0,0,10.0,11.0,"[[[0.08202077598288503, -0.02283112974341891],...",1,3,8,uniform,uniform,win,741,1802,0.250,win
2,0,0,10.5,11.5,"[[[0.08202077598288503, -0.02283112974341891],...",1,2,3,uniform,uniform,win,482,1432,0.125,win
3,1,1,10.0,11.0,"[[[0.08202077598288503, -0.02283112974341891],...",1,5,6,uniform,uniform,lose,1356,1901,0.500,lose
4,0,0,9.5,10.5,"[[[0.08202077598288503, -0.02283112974341891],...",1,3,2,uniform,uniform,lose,480,1454,0.250,lose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,1,1,33.0,67.0,"[[[0.47130341854314806, -0.02283112974341891],...",4,8,7,high,mix,win,1218,1400,0.243,win
266,1,1,33.5,67.5,"[[[0.47130341854314806, -0.02283112974341891],...",4,9,6,high,mix,win,509,1535,0.000,win
267,0,0,34.0,68.0,"[[[0.47130341854314806, -0.02283112974341891],...",4,3,4,high,mix,win,844,1350,0.071,win
268,1,1,33.5,67.5,"[[[0.47130341854314806, -0.02283112974341891],...",4,5,8,uniform,mix,lose,870,1325,0.500,lose


# create data

In [6]:
final_dfs = []

for idx, name in enumerate(clean_names):
    participant_df = df_participants[idx]
    greedy_df = df_both_greedy[idx]
    softmax_df = df_both_softmax[idx]
    wsls_df = df_both_wsls[idx]
    rs_df = df_both_rs[idx]
    dualQ_df = df_both_dualQ[idx]
    dualQ_no_eta_df = df_both_dualQ_no_eta[idx]

    filtered_rows = []

    for _, row in common_trials.iterrows():
        myCard = row['myCard']
        dist = row['distribution']
        n = int(row['min_count'])

        mask = (participant_df['myCard'] == myCard) & (participant_df['distribution'] == dist)
        selected_indices = participant_df[mask].index[:n]

        for i in selected_indices:
            filtered_rows.append({
                'myCard': myCard,
                'distribution': dist,
                'model_choice_greedy': greedy_df.loc[i, 'model_choices'],
                'model_choice_softmax': softmax_df.loc[i, 'model_choices'],
                'model_choice_wsls': wsls_df.loc[i, 'model_choices'],
                'model_choice_rs': rs_df.loc[i, 'model_choices'],
                'model_choice_dualQ': dualQ_df.loc[i, 'model_choices'],
                'model_choice_dualQ_no_eta': dualQ_no_eta_df.loc[i, 'model_choices'],
                'participant_choice': greedy_df.loc[i, 'participant_choices'],
                'model_outcome_greedy': greedy_df.loc[i, 'model_outcome'],
                'model_outcome_softmax': softmax_df.loc[i, 'model_outcome'],
                'model_outcome_wsls': wsls_df.loc[i, 'model_outcome'],
                'model_outcome_rs': rs_df.loc[i, 'model_outcome'],
                'model_outcome_dualQ': dualQ_df.loc[i, 'model_outcome'],
                'model_outcome_dualQ_no_eta': dualQ_no_eta_df.loc[i, 'model_outcome'],
                'arrowRT': participant_df.loc[i, 'arrowRT'],
                'spaceRT': participant_df.loc[i, 'spaceRT'],
                'risk': participant_df.loc[i, 'risk'],
                'block_type': participant_df.loc[i, 'block_type'],
                'participant_outcome': participant_df.loc[i, 'outcome']
            })

    result_df = pd.DataFrame(filtered_rows)
    final_dfs.append(result_df)

    # Save each result
    output_path = os.path.join(output_dir, f"{name}.csv")
    result_df.to_csv(output_path, index=False)
